In [ ]:
# 런타임 리셋 : 세션 초기화됩니다. 주의!
import os
os.kill(os.getpid(), 9)

## README



```
> 테스트 모델 :
    - naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B

> 방식
    - Full-fineTunning
    - LoRA

> 허깅페이스 아이디
    - haebo

```



## Model Load

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from google.colab import userdata

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HF_TOKEN = userdata.get('HF_TOKEN')
print(f'device : {DEVICE}')
print(f'HF_TOKEN : {True if HF_TOKEN else False}')
print(f"GPU : {torch.cuda.get_device_name(0)}")

device : cuda
HF_TOKEN : True
GPU : NVIDIA A100-SXM4-40GB


In [ ]:
model_id = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
print(f'model : {model_id}')
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map="auto", # 모델 연산 CPU 분산
    token=HF_TOKEN
)

model : naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B


In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(110592, 2048, padding_idx=100257)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (up_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (down_proj): Linear(in_features=7168, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e

In [ ]:
!nvidia-smi
!free -h

Tue May 20 06:25:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             50W /  400W |    7415MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Full FineTunning

Full Fine-tuning은 사전학습된 모델의 모든 파라미터를 학습 가능한 상태로 열고, 주어진 데이터셋을 통해 모델 전체를 미세조정하는 방식입니다. 가장 강력한 튜닝 방식이지만, 메모리 사용량과 학습 시간이 많아 Colab에서는 batch size = 1 또는 gradient accumulation이 필요할 수 있습니다.

In [ ]:
from huggingface_hub import HfApi, login

# 1. 로그인 (토큰 파일에 저장)
login(token=HF_TOKEN)

# 2. API 객체로 로그인 상태 확인
api = HfApi()
print("✅ 현재 사용자:", api.whoami()["name"])

✅ 현재 사용자: haebo


### 기본 준비

In [ ]:
!pwd
!rm -rf ~/.cache/huggingface/datasets

/content


In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset, Features, Value
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import shutil
import os

# 1. 파일 경로
file_path = "./posts-0515.jsonl"

# 2. 데이터 구조 명시 (features)
features = Features({
    "content": Value("string"),
    "emotion": Value("string"),
    "post_type": Value("string"),
    "transformed_content": Value("string")
})

# 3. 임시 캐시 디렉토리 설정 (삭제 예정)
temp_cache_dir = "./tmp_cache"
os.makedirs(temp_cache_dir, exist_ok=True)

# 4. 데이터셋 로드
dataset = load_dataset(
    "json",
    data_files=file_path,
    features=features,
    split="train",
    cache_dir=temp_cache_dir,
    verification_mode="no_checks"  # ⚠️ ignore_verifications는 deprecated
)

# 5. 캐시 디렉터리 삭제 (완전한 캐시 제거)
shutil.rmtree(temp_cache_dir, ignore_errors=True)

# ✅ 결과 확인
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'content': 's3 이미지 url 잘 변경됐나', 'emotion': 'happy', 'post_type': 'dog', 'transformed_content': '어흥! 🐶 s3 이미지 url, 완전 잘 변경됐는지 궁금해! 내가 열심히 지켜볼게! 💖 혹시 문제 생기면 바로 짖을게! 멍! ☀️'}


In [ ]:
def format_and_tokenize(example):
    text = (
        f"### 사용자:\n{example['content']}\n"
        f"### 감정:\n{example['emotion']}\n"
        f"### 동물:\n{example['post_type']}\n\n"
        f"### 말투 변환 결과:\n{example['transformed_content']}"
    )
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(format_and_tokenize, remove_columns=dataset.column_names)
tokenized_dataset

Map:   0%|          | 0/342 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 342
})

In [ ]:
print(next(model.parameters()).device)
print(next(model.parameters()).dtype)

cuda:0
torch.float32


### 파인튜닝 진행

In [ ]:
import torch
import gc

torch.cuda.empty_cache()  # 사용하지 않는 메모리를 GPU에서 해제
gc.collect()              # Python garbage collector 강제 실행

140

In [ ]:
# ======================================================
# ✅ 학습 설정 및 Trainer 구성
# ======================================================
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

output_dir = "./finetuned-ktb"
repo_name = "Meow-HyperCLOVAX-1.5B-FullFT-fp32"
repo_id = f"haebo/{repo_name}"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    fp16=False,
    bf16=False,
    logging_steps=1,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    push_to_hub=True,
    hub_model_id=repo_id,
    hub_strategy="end"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# ======================================================
# ✅ 학습 실행
# ======================================================
trainer.train()

<ipython-input-12-bc8c440cdcb6>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,3.957500
2,3.398000
3,2.652500
4,2.480100
5,2.336300
6,2.028600
7,1.796800
8,2.141300
9,2.058800
10,2.435400


TrainOutput(global_step=210, training_loss=0.7355340094083831, metrics={'train_runtime': 887.2002, 'train_samples_per_second': 1.927, 'train_steps_per_second': 0.237, 'total_flos': 6980627576389632.0, 'train_loss': 0.7355340094083831, 'epoch': 4.888888888888889})

### 모델 저장

In [ ]:
# ======================================================
# ✅ 모델 저장 (로컬)
# ======================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

model-00002-of-00002.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

('./finetuned-ktb/tokenizer_config.json',
 './finetuned-ktb/special_tokens_map.json',
 './finetuned-ktb/vocab.json',
 './finetuned-ktb/merges.txt',
 './finetuned-ktb/added_tokens.json',
 './finetuned-ktb/tokenizer.json')

In [ ]:
# ======================================================
# ✅ Hugging Face Hub에 업로드 (Model Hub 전용)
# ======================================================
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"✅ 모델 업로드 완료: https://huggingface.co/{repo_id}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


✅ 모델 업로드 완료: https://huggingface.co/haebo/Meow-HyperCLOVAX-1.5B-FullFT-fp32


In [ ]:
!nvidia-smi
!free -h

Tue May 20 04:51:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             50W /  400W |   33327MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### 모델 로드 및 추론

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("haebo/Meow-HyperCLOVAX-1.5B-FullFT-fp32")
model = AutoModelForCausalLM.from_pretrained("haebo/Meow-HyperCLOVAX-1.5B-FullFT-fp32")

tokenizer_config.json:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# prompt: 모델로 답변 추론하는 코드
model.eval()
model.to(DEVICE)

# ## 모델 추론
# ======================================================
# ✅ 추론 함수
# ======================================================

def generate_response(prompt, max_length=100):
    # 프롬프트 템플릿 적용
    input_text = (
    f"### 사용자:\n{prompt}\n"
    f"### 감정:\n{emotion}\n"
    f"### 동물:\n{post_type}\n\n"
    f"### 말투 변환 결과:"
)

    # 토크나이징
    inputs = tokenizer(input_text, return_tensors="pt").to(DEVICE)

    # 모델 추론
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7, # 샘플링 온도 (높을수록 다양성 증가)
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1 # 생성할 시퀀스 수
        )

    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 사용자 입력 부분 제거 (템플릿에 맞춰서)
    # ### 사용자:\n{prompt}\n\n### 고양이: 이 뒤의 내용만 추출
    start_index = response.find("### 고양이:")
    if start_index != -1:
        response = response[start_index + len("### 고양이:"):].strip()

    return response

# ======================================================
# ✅ 추론 테스트
# ======================================================

prompt1 = "오늘 저녁은 뭐 먹지?"
response1 = generate_response(prompt1)
print(f"사용자: {prompt1}")
print(f"고양이: {response1}")

print("-" * 30)

prompt2 = "날씨가 춥다."
response2 = generate_response(prompt2)
print(f"사용자: {prompt2}")
print(f"고양이: {response2}")

print("-" * 30)

prompt3 = "산책 가고 싶다."
response3 = generate_response(prompt3)
print(f"사용자: {prompt3}")
print(f"고양이: {response3}")

사용자: 오늘 저녁은 뭐 먹지?
고양이: 냐아~🐾 오늘 저녁엔 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~🐾🐾🐾🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~🐾🐾🐾🐾🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~�
------------------------------
사용자: 날씨가 춥다.
고양이: 냐아아~🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~😻🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~😻�
------------------------------
사용자: 산책 가고 싶다.
고양이: 냐아아~🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹~🐾🐾 오늘은 창밖 햇살이 털 사이로 스며드는 날이다옹!🐾🐾 오늘은 창밖 햇살이 털 사이로 스
